# NPRI Time Series Analysis - Data Preprocessing

This notebook focuses on preprocessing the National Pollutant Release Inventory (NPRI) dataset merged with 2023 data from the CMPT2400 project. The preprocessing steps prepare the data for time series analysis and forecasting.

## Overview

The preprocessing pipeline includes:

1. Data cleaning and handling missing values
2. Feature engineering for time series analysis
3. Creating lag variables
4. One-hot encoding categorical variables
5. Preparing train/test splits for model evaluation

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import os

# Import utility functions (optional)
import sys
sys.path.append('..')
from src.utils.data_utils import load_data, create_lag_features, one_hot_encode_categories

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Load and Initial Inspection

First, let's load the dataset that was previously examined in the exploration notebook.

In [ ]:
# Load the dataset
df_releases = pd.read_csv('../data/raw/df_merged_releases.csv')

# Display basic information
print(f"Dataset shape: {df_releases.shape}")
print("\nFirst 5 rows:")
df_releases.head()

## 2. Data Cleaning

Let's clean the dataset by handling missing values, removing duplicates, and addressing any data quality issues identified in the exploration phase.

In [ ]:
# Check for missing values
missing_values = df_releases.isnull().sum()
print("Missing values:")
print(missing_values[missing_values > 0])

# Handle missing values
df_cleaned = df_releases.copy()

# Fill missing values for release columns with 0 (assuming missing means no release)
release_columns = ['Total_Air_Releases', 'Total_Land_Releases', 'Total_Water_Releases']
for col in release_columns:
    df_cleaned[col] = df_cleaned[col].fillna(0)

# Handle other missing values as appropriate
# [Add additional missing value handling logic based on exploration findings]

# Drop duplicates if any
df_cleaned = df_cleaned.drop_duplicates()

# Verify cleaning results
print(f"\nDataset shape after cleaning: {df_cleaned.shape}")
print("\nMissing values after cleaning:")
print(df_cleaned.isnull().sum()[df_cleaned.isnull().sum() > 0])

## 3. Feature Engineering

Now, let's create features that will be useful for time series modeling, including lag features and time-based features.

In [ ]:
# Create lag features for each release type
df_with_lags = df_cleaned.copy()

# Define grouping columns for creating lag features
# (e.g., create lags within each facility-substance combination)
group_cols = ['Facility_Name', 'Substance_Name', 'Industry_Sector']

# Create lag features for each release type
for col in release_columns:
    df_with_lags = create_lag_features(df_with_lags, col, lag_periods=7, group_cols=group_cols)

# Display the dataframe with lag features
print("\nDataframe with lag features:")
df_with_lags.head()

In [ ]:
# Create additional time-based features

# Add year difference from baseline
min_year = df_with_lags['Reporting_Year'].min()
df_with_lags['Year_Diff'] = df_with_lags['Reporting_Year'] - min_year

# Create cyclical features if there are seasonal patterns
# (Not applicable if data is annual, but included as an example)
if 'Month' in df_with_lags.columns:
    df_with_lags['Month_Sin'] = np.sin(2 * np.pi * df_with_lags['Month'] / 12)
    df_with_lags['Month_Cos'] = np.cos(2 * np.pi * df_with_lags['Month'] / 12)

# Display added features
print("\nDataframe with time-based features:")
df_with_lags.head()

## 4. Categorical Variable Encoding

Next, let's encode categorical variables such as Industry_Sector.

In [ ]:
# Identify categorical columns
categorical_cols = ['Industry_Sector'] 
# Add other categorical columns if needed

# One-hot encode categorical variables
df_encoded = one_hot_encode_categories(df_with_lags, categorical_cols)

# Display encoded dataframe
print("\nDataframe with one-hot encoded categories:")
df_encoded.head()

## 5. Time Series Split Preparation

For time series modeling, we need to create train/test splits that respect the chronological order of the data.

In [ ]:
# Sort data chronologically
df_encoded = df_encoded.sort_values(by=['Reporting_Year']).reset_index(drop=True)

# Create train/test split (80% train, 20% test)
split_idx = int(len(df_encoded) * 0.8)
train_df = df_encoded.iloc[:split_idx].copy()
test_df = df_encoded.iloc[split_idx:].copy()

print(f"\nTrain set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")

# Verify year ranges in train and test sets
print(f"\nTrain set year range: {train_df['Reporting_Year'].min()} - {train_df['Reporting_Year'].max()}")
print(f"Test set year range: {test_df['Reporting_Year'].min()} - {test_df['Reporting_Year'].max()}")

## 6. Prepare Data for Modeling

Finally, let's format the data to make it ready for the modeling phase. We'll create specific datasets for each target variable.

In [ ]:
# Function to prepare data for a specific target variable
def prepare_for_modeling(df, target_col, lag_cols):
    """
    Prepare data for modeling by selecting appropriate features and target.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe
    target_col : str
        Target column name
    lag_cols : list
        List of lag column names to use as features
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame ready for modeling
    """
    # Get categorical columns (one-hot encoded industry sectors)
    industry_cols = [col for col in df.columns if col.startswith('Industry_Sector_')]
    
    # Select features and target
    feature_cols = ['Reporting_Year', 'Year_Diff'] + lag_cols + industry_cols
    
    # Create modeling dataframe
    model_df = df[feature_cols + [target_col]].copy()
    
    # Drop rows with missing lag values
    model_df = model_df.dropna()
    
    return model_df

In [ ]:
# Prepare modeling datasets for each release type
modeling_dfs = {}

for target_col in release_columns:
    # Get lag columns for this target
    lag_cols = [f"{target_col}_lag_{i}" for i in range(1, 8)]
    
    # Prepare train and test datasets
    train_model_df = prepare_for_modeling(train_df, target_col, lag_cols)
    test_model_df = prepare_for_modeling(test_df, target_col, lag_cols)
    
    # Store in dictionary
    modeling_dfs[target_col] = {
        'train': train_model_df,
        'test': test_model_df
    }
    
    print(f"\nPrepared {target_col} datasets:")
    print(f"Train shape: {train_model_df.shape}")
    print(f"Test shape: {test_model_df.shape}")

## 7. Save Processed Data

Finally, let's save the processed datasets for use in the modeling notebook.

In [ ]:
# Create processed data directory if it doesn't exist
processed_dir = '../data/processed'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Save each processed dataset
for target_col, datasets in modeling_dfs.items():
    # Create target-specific filenames
    train_filename = f"{target_col.lower().replace('_', '-')}_train.csv"
    test_filename = f"{target_col.lower().replace('_', '-')}_test.csv"
    
    # Save train and test datasets
    datasets['train'].to_csv(os.path.join(processed_dir, train_filename), index=False)
    datasets['test'].to_csv(os.path.join(processed_dir, test_filename), index=False)
    
    print(f"Saved {target_col} datasets to {processed_dir}")

## 8. Summary

In this notebook, we performed the following preprocessing steps:

1. Data cleaning: handled missing values and removed duplicates
2. Feature engineering: created lag variables and time-based features
3. Categorical encoding: one-hot encoded industry sectors
4. Time series split: created chronologically ordered train/test sets
5. Modeling preparation: created specific datasets for each target variable

The processed data is now ready for modeling in the next notebook (03_modeling_evaluation.ipynb).